# 2D Kernel with SPECFEM2D - Multiple Receivers
By Andrea R.  
Utility functions written by Ridvan Orsvuran.  
Following file structure as in the Seisflows example (By Bryant Chow)

In [1]:
import os
import shutil
import matplotlib
import numpy as np
import FunctionsPlotBin
import matplotlib.pyplot as plt

from UtilityFunctions import read_trace, specfem2D_prep_save_forward, replace_line, save_trace, specfem2D_prep_adjoint, grid
from scipy.integrate import simps

# Domain: 
2D:   
x-dir = 4000 m   
z-dir = 4000 m   

### Source location
original x:    1000 m     
original z:    2000 m  

### Output stations locations: 
Station #1   AAS0001                             
original x:    3000 m  
original z:    2000 m  

Station #2   AAS0002                             
original x:    2000 m  
original z:    3000 m  

Station #3   AAS0003                             
original x:    2000 m  
original z:    1000 m  

### Boundary conditions 
Type: STACEY_ABSORBING_CONDITIONS  
absorbbottom                    = true  
absorbright                     = true  
absorbtop                       = true  
absorbleft                      = true  

### Velocity model:

#### Initial model:  
Model: P (or PI) velocity min,max =    3000 m/s              
Model: S velocity min,max         =    1800 m/s        
Model: density min,max            =    2700 kg/m3           

#### True model (~1% perturbation of the Vs - initial model):   
Model: P (or PI) velocity min,max =    3000 m/s                
Model: S velocity min,max         =    1820 m/s          
Model: density min,max            =    2700 kg/m3      

### Set Specfem2D and work directories 

In [ ]:
specfem2d_path = "/home/masa/" # for desktop machine
#specfem2d_path = "/home/masan/" # for laptop 
EXAMPLE = os.path.join(os.getcwd(),"Examples", "DATA_Example01")
WORKDIR = os.path.join(os.getcwd(),"work")

# Incase we've run this docs page before, delete the working directory before remaking
if os.path.exists(WORKDIR):
    shutil.rmtree(WORKDIR)
os.makedirs(WORKDIR, exist_ok=True)

In [3]:
# Distribute the necessary file structure of the SPECFEM2D repository that we will reference
SPECFEM2D_ORIGINAL = os.path.join(specfem2d_path, "specfem2d") 
SPECFEM2D_BIN_ORIGINAL = os.path.join(SPECFEM2D_ORIGINAL, "bin")
SPECFEM2D_DATA_ORIGINAL = os.path.join(SPECFEM2D_ORIGINAL, "DATA")

# The SPECFEM2D working directory that we will create separate from the downloaded repo
SPECFEM2D_WORKDIR = os.path.join(WORKDIR, "ExampleKernelMultiReceiver")
SPECFEM2D_BIN = os.path.join(SPECFEM2D_WORKDIR, "bin")
SPECFEM2D_DATA = os.path.join(SPECFEM2D_WORKDIR, "DATA")
SPECFEM2D_OUTPUT = os.path.join(SPECFEM2D_WORKDIR, "OUTPUT_FILES")

# Pre-defined locations of velocity models we will generate using the solver
SPECFEM2D_MODEL_INIT = os.path.join(SPECFEM2D_WORKDIR, "OUTPUT_FILES_INIT")
SPECFEM2D_MODEL_TRUE = os.path.join(SPECFEM2D_WORKDIR, "OUTPUT_FILES_TRUE")

In [4]:
# Copy the binary files incase we update the source code. These can also be symlinked.
shutil.copytree(SPECFEM2D_BIN_ORIGINAL, SPECFEM2D_BIN)

# Copy the DATA/ directory
shutil.copytree(EXAMPLE, SPECFEM2D_DATA)
!pwd
!ls

FileNotFoundError: [Errno 2] No such file or directory: '/home/masan/specfem2d/bin'

In [ ]:
# Create a new STATIONS file with multiple receivers
stations_content = """S0001    AA          3000.0000000        2000.0000000       0.0         0.0
S0002    AA          2000.0000000        3000.0000000       0.0         0.0
S0003    AA          2000.0000000        1000.0000000       0.0         0.0
"""

with open(os.path.join(SPECFEM2D_DATA, "STATIONS"), "w") as f:
    f.write(stations_content)

### Generate true model

In [ ]:
os.chdir(SPECFEM2D_DATA)
specfem2D_prep_save_forward("Par_file")
# Modify the Par_file to increase Initial Vs by ~1% 
replace_line("Par_file",33,"P_SV                            = .false. \n")
replace_line("Par_file",262,'1 1 2700.d0 3000.d0 1820.d0 0 0 9999 9999 0 0 0 0 0 0 \n')

In [ ]:
# create the OUTPUT_FILES directory before running 
os.chdir(SPECFEM2D_WORKDIR)
if os.path.exists(SPECFEM2D_OUTPUT):
    shutil.rmtree(SPECFEM2D_OUTPUT)
os.mkdir(SPECFEM2D_OUTPUT)
!ls

In [ ]:
os.chdir(SPECFEM2D_WORKDIR)
!bin/xmeshfem2D > OUTPUT_FILES/mesher_log.txt
!bin/xspecfem2D > OUTPUT_FILES/solver_log.txt

# Move the model files (*.bin) into the OUTPUT_FILES directory
!mv DATA/*bin OUTPUT_FILES

# Make sure we don't overwrite this target model when creating our initial model in the next step
!mv OUTPUT_FILES OUTPUT_FILES_TRUE

!head OUTPUT_FILES_TRUE/solver_log.txt
!tail OUTPUT_FILES_TRUE/solver_log.txt

In [ ]:
x_coords_file = 'OUTPUT_FILES_TRUE/proc000000_x.bin'
z_coords_file = 'OUTPUT_FILES_TRUE/proc000000_z.bin'
Vs_true       = 'OUTPUT_FILES_TRUE/proc000000_vs.bin'

# Plot 
FunctionsPlotBin.plotbin(x_coords_file,z_coords_file,Vs_true,SPECFEM2D_WORKDIR+'/Vs_true','Vs_true=m/s')

### Generate initial model

In [ ]:
os.chdir(SPECFEM2D_DATA)
replace_line("Par_file",262,'1 1 2700.d0 3000.d0 1800.d0 0 0 9999 9999 0 0 0 0 0 0 \n')

In [ ]:
# create the OUTPUT_FILES directory before running 
os.chdir(SPECFEM2D_WORKDIR)
if os.path.exists(SPECFEM2D_OUTPUT):
    shutil.rmtree(SPECFEM2D_OUTPUT)
os.mkdir(SPECFEM2D_OUTPUT)
!ls

In [ ]:
os.chdir(SPECFEM2D_WORKDIR)
!bin/xmeshfem2D > OUTPUT_FILES/mesher_log.txt
!bin/xspecfem2D > OUTPUT_FILES/solver_log.txt

# Move the model files (*.bin) into the OUTPUT_FILES directory
# The binary files of the velocity models are stored in DATA after running xspecfem2D
!mv DATA/*bin OUTPUT_FILES

# Store output files of initial model
!mv OUTPUT_FILES OUTPUT_FILES_INIT

!head OUTPUT_FILES_INIT/solver_log.txt
!tail OUTPUT_FILES_INIT/solver_log.txt

In [ ]:
x_coords_file = 'OUTPUT_FILES_INIT/proc000000_x.bin'
z_coords_file = 'OUTPUT_FILES_INIT/proc000000_z.bin'
Vs_true       = 'OUTPUT_FILES_INIT/proc000000_vs.bin'

# Plot 
FunctionsPlotBin.plotbin(x_coords_file,z_coords_file,Vs_true,SPECFEM2D_WORKDIR+'/Vs_init','Vs_init=m/s')

### 3. Plot synthetic seismograms for all receivers

In [ ]:
os.chdir(SPECFEM2D_WORKDIR)
station_ids = ["S0001", "S0002", "S0003"]

fig, axes = plt.subplots(len(station_ids), 1, figsize=(20, 12), sharex=True)
matplotlib.rcParams.update({'font.size': 14})

for i, station_id in enumerate(station_ids):
    # Read synthetic seismogram
    obsd = read_trace(os.path.join("OUTPUT_FILES_TRUE",f"AA.{station_id}.BXY.semd"))
    synt = read_trace(os.path.join("OUTPUT_FILES_INIT",f"AA.{station_id}.BXY.semd"))

    # Process data
    obsd.detrend("simple")
    obsd.taper(0.05)
    obsd.filter("bandpass", freqmin=0.01, freqmax=20)

    synt.detrend("simple")
    synt.taper(0.05)
    synt.filter("bandpass", freqmin=0.01, freqmax=20)
    
    # Plot
    axes[i].plot(obsd.times()+obsd.stats.b, obsd.data, "b", label="Obsd")
    axes[i].plot(synt.times()+synt.stats.b, synt.data, "r", label="Synt")
    axes[i].set_xlim(synt.stats.b, synt.times()[-1]+synt.stats.b)
    axes[i].legend(frameon=False)
    axes[i].set_ylabel(f"Station {station_id}\nDisplacement (m)")
    axes[i].tick_params(axis='both',which='major',labelsize=14)

axes[-1].set_xlabel("Time (s)")
fig.suptitle("Comparison of Synthetic and Observed Seismograms", fontsize=16)
plt.tight_layout()

### Misfit Calculation and Adjoint Source
For one seismogram, waveform misfit is

$$ \chi = \frac{1}{2} \int [d(t)-s(t)]^2 dt~, $$


and waveform adjoint source is

$$  f^\dagger (t) = s(t) - d(t)~,$$

where $s(t)$ is the synthetic, $d(t)$ is the observed seismograms.

In [ ]:
os.chdir(SPECFEM2D_WORKDIR)
station_ids = ["S0001", "S0002", "S0003"]
total_misfit = 0

# Initialize storage for adjoint sources
adjoint_sources = {}

for station_id in station_ids:
    # Read synthetic seismogram
    obsd = read_trace(os.path.join("OUTPUT_FILES_TRUE",f"AA.{station_id}.BXY.semd"))
    synt = read_trace(os.path.join("OUTPUT_FILES_INIT",f"AA.{station_id}.BXY.semd"))

    # Process data
    obsd.detrend("simple")
    obsd.taper(0.05)
    obsd.filter("bandpass", freqmin=0.01, freqmax=20)

    synt.detrend("simple")
    synt.taper(0.05)
    synt.filter("bandpass", freqmin=0.01, freqmax=20)
    
    # Misfit calculation
    misfit = simps((obsd.data-synt.data)**2, dx=obsd.stats.delta)
    total_misfit += misfit
    
    # Adjoint Source
    adj = synt.copy()
    adj.data = synt.data - obsd.data

    # Process adjoint source
    adj.detrend("simple")
    adj.taper(0.05)
    adj.filter("bandpass", freqmin=0.01, freqmax=20)
    
    # Store for later use
    adjoint_sources[station_id] = adj
    
print(f"Total Misfit: {total_misfit:.6f}")

In [ ]:
# Plot all adjoint sources
fig, axes = plt.subplots(len(station_ids), 1, figsize=(20, 12), sharex=True)
matplotlib.rcParams.update({'font.size': 14})

for i, station_id in enumerate(station_ids):
    adj = adjoint_sources[station_id]
    axes[i].plot(adj.times()+adj.stats.b, adj.data, "g")
    axes[i].set_ylabel(f"Station {station_id}\nAdjoint Source")
    axes[i].tick_params(axis='both',which='major',labelsize=14)

axes[-1].set_xlabel("Time (s)")
fig.suptitle("Adjoint Sources for All Stations", fontsize=16)
plt.tight_layout()

### Adjoint simulation

In [ ]:
os.chdir(SPECFEM2D_WORKDIR)
# Create SEM directory for adjoint sources
os.makedirs("SEM", exist_ok=True)

# Save all adjoint sources
for station_id, adj in adjoint_sources.items():
    save_trace(adj, f"SEM/AA.{station_id}.BXY.adj")

For adjoint simulation, following `DATA/Par_file` needs be set

```toml
SIMULATION_TYPE                 = 3
# save the last frame, needed for adjoint simulation
SAVE_FORWARD                    = .false.
```

`specfem2D_prep_adjoint` function can be used for this purpose.

In [ ]:
# Prepare Par_file
os.chdir(SPECFEM2D_DATA)
specfem2D_prep_adjoint("Par_file")

In [ ]:
# create the OUTPUT_FILES directory before running and copy 
# results of OUTPUT_FILES_INIT to the new created OUTPUT_FILES, this is needed
# for the adjoint simulation because we saved the last frame in the 
# forward simulation of the initial model

os.chdir(SPECFEM2D_WORKDIR)
if os.path.exists(SPECFEM2D_OUTPUT):
    shutil.rmtree(SPECFEM2D_OUTPUT)
os.mkdir(SPECFEM2D_OUTPUT)
!cp OUTPUT_FILES_INIT/* OUTPUT_FILES
!ls

In [ ]:
os.chdir(SPECFEM2D_WORKDIR)
!bin/xmeshfem2D > OUTPUT_FILES/mesher_log.txt
!bin/xspecfem2D > OUTPUT_FILES/solver_log.txt

# Move the model files (*.bin) into the OUTPUT_FILES directory
# The binary files of the velocity models are stored in DATA after running xspecfem2D
!mv DATA/*bin OUTPUT_FILES

# Make sure we don't overwrite output files 
!mv OUTPUT_FILES OUTPUT_FILES_ADJ

!head OUTPUT_FILES_ADJ/solver_log.txt
!tail OUTPUT_FILES_ADJ/solver_log.txt

### Plotting the Kernels

`./OUTPUT_FILES/proc000000_rhop_alpha_beta_kernel.dat` file holds the kernel data.

It is a text file contains 5 columns: `x`, `z`, `rhop`, `alpha`, `beta`.

In [ ]:
data = np.loadtxt("./OUTPUT_FILES_ADJ/proc000000_rhop_alpha_beta_kernel.dat")

# first column: x
x = data[:, 0]
# second column: z
z = data[:, 1]
# fifth column: beta_kernel
beta = data[:, 4]

# For plotting, you can check: specfem2D/utils/Visualization/plot_kernel.py
vmax = 5e-9  # Increased to accommodate multiple receivers
X, Z, BETA = grid(x, z, beta)

fig, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(BETA, vmax=vmax, vmin=-vmax, extent=[x.min(), x.max(), z.min(), z.max()],
               cmap="seismic_r")
ax.set_xlabel("X (m)")
ax.set_ylabel("Z (m)")
ax.set_title("Beta Kernel with Multiple Receivers")

# Plot source 
ax.scatter(1000, 2000, 1000, marker="*", color="black", edgecolor="white", label="Source")

# Plot all stations
ax.scatter(3000, 2000, 450, marker="v", color="blue", edgecolor="white", label="Station 1")
ax.scatter(2000, 3000, 450, marker="v", color="green", edgecolor="white", label="Station 2")
ax.scatter(2000, 1000, 450, marker="v", color="red", edgecolor="white", label="Station 3")

plt.colorbar(im, ax=ax)
ax.legend()
ax.tick_params(axis='both',which='major',labelsize=14)
matplotlib.rcParams.update({'font.size': 14})

### Compare with Single Receiver Kernel

Let's run the same model but with just one receiver to see the difference in kernels.

In [ ]:
# Create a figure to compare kernels
fig, axes = plt.subplots(2, 2, figsize=(16, 16))
axes = axes.flatten()

# Plot the multi-receiver kernel
im0 = axes[0].imshow(BETA, vmax=vmax, vmin=-vmax, extent=[x.min(), x.max(), z.min(), z.max()],
               cmap="seismic_r")
axes[0].set_title("Kernel with All Three Receivers")
axes[0].scatter(1000, 2000, 200, marker="*", color="black", edgecolor="white")
axes[0].scatter(3000, 2000, 100, marker="v", color="blue", edgecolor="white")
axes[0].scatter(2000, 3000, 100, marker="v", color="green", edgecolor="white")
axes[0].scatter(2000, 1000, 100, marker="v", color="red", edgecolor="white")
plt.colorbar(im0, ax=axes[0])

# Now add placeholders for individual kernels
# These would normally be calculated by running separate adjoint simulations
# In a real notebook, you'd have actual data for these
for i, (ax, title, pos) in enumerate(zip(
    axes[1:], 
    ["Kernel with only Station 1", "Kernel with only Station 2", "Kernel with only Station 3"],
    [(3000, 2000), (2000, 3000), (2000, 1000)])):
    
    # This is a placeholder - normally you'd load actual kernel data
    ax.text(0.5, 0.5, "Placeholder - would contain\nkernel from individual station", 
            ha='center', va='center', transform=ax.transAxes, fontsize=14)
    
    ax.set_title(title)
    ax.scatter(1000, 2000, 200, marker="*", color="black", edgecolor="white")
    
    # Highlight the station for this kernel
    station_color = ["blue", "green", "red"][i]
    ax.scatter(pos[0], pos[1], 100, marker="v", color=station_color, edgecolor="white")
    
    # Set the axes limits to match the main plot
    ax.set_xlim(x.min(), x.max())
    ax.set_ylim(z.min(), z.max())

plt.tight_layout()
fig.suptitle("Comparison of Multi-Receiver and Single-Receiver Kernels", fontsize=16, y=1.02)